### PyEmma Featurizer Support

In [1]:
import openpathsampling as paths
import numpy as np

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
#! lazy
ref_storage = paths.Storage('engine_store_test.nc', mode='r')

03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Open existing netCDF file 'engine_store_test.nc' for reading - reading from existing file


In [4]:
#! lazy
storage = paths.Storage('delete.nc', 'w')
storage.trajectories.save(ref_storage.trajectories[0])

03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Create new netCDF file 'delete.nc' for writing - deleting existing file
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Setup netCDF file and create variables
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'trajectories'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'topologies'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'cvs'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'snapshots'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'samples'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'samplesets'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing store 'movechanges'
03-08-16 13:18:42 openpathsampling.netcdfplus.netcdfplus INFO     Initializing 

UUID('e1a53e23-58b2-11e6-8656-000000000014')

Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [5]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [6]:
cv = paths.collectivevariable.PyEMMAFeaturizerCV(
    'pyemma', 
    pyemma_generator, 
    topology=ref_storage.snapshots[0].topology
).with_diskcache()

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [7]:
cv(ref_storage.trajectories[0]);

Let's save it to the storage

In [8]:
#! lazy
print storage.save(cv)

(store.cvs[CollectiveVariable], 2, UUID('07f69605-596c-11e6-9658-0000000000ac'))


and apply the featurizer to a trajectory

In [9]:
cv(storage.trajectories[0]);

Sync to make sure the cache is written to the netCDF file.

In [10]:
cv(storage.snapshots.all());

In [11]:
py_cv = storage.cvs['pyemma']

In [12]:
store = storage.stores['cv%d' % storage.idx(py_cv)]
nc_var = store.variables['value']

In [13]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [14]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [15]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "PyEMMAFeaturizerCV", "_dict": {"topology": {"_obj": "topologies", "_uuid": "e1a53e23-58b2-11e6-8656-000000000002"}, "name": "pyemma", "featurizer": {"_marshal": "YwEAAAABAAAAAwAAAEMAAABzIAAAAHwAAGoAAHwAAGoBAHwAAGoCAIMAAIMBAIMBAAFkAABTKAEAAABOKAMAAAB0FQAAAGFkZF9pbnZlcnNlX2Rpc3RhbmNlc3QFAAAAcGFpcnN0DwAAAHNlbGVjdF9CYWNrYm9uZSgBAAAAdAEAAABmKAAAAAAoAAAAAHMeAAAAPGlweXRob24taW5wdXQtNS00Nzc0ZDhlZGRkMDA+dBAAAABweWVtbWFfZ2VuZXJhdG9yAQAAAHMCAAAAAAE=", "_module_vars": [], "_global_vars": []}, "kwargs": {}}}']


In [16]:
py_cv_idx = storage.idx(py_cv)
print py_cv_idx
py_emma_feat = storage.vars['cvs_json'][py_cv_idx]

0


In [17]:
erg = py_emma_feat(storage.snapshots);

In [18]:
#! lazy
print erg[0::5,2:4]

[[ 2.68972969  2.06547379]
 [ 2.61396885  2.00944018]
 [ 2.73087001  2.06900215]
 [ 2.79462361  2.09850192]]


In [19]:
storage.close()
ref_storage.close()

In [20]:
#! lazy
storage = paths.Storage('delete.nc', 'r')

03-08-16 13:18:54 openpathsampling.netcdfplus.netcdfplus INFO     Open existing netCDF file 'delete.nc' for reading - reading from existing file


In [21]:
cv = storage.cvs[0]

Make sure that we get the same result

In [22]:
assert np.allclose(erg, cv(storage.snapshots))

In [23]:
storage.close()